In [ ]:
# 1. Clone PiCO Github Repo files onto Colab Session

!git clone https://github.com/PKU-YuanGroup/Peer-review-in-LLMs.git
%cd Peer-review-in-LLMs

In [ ]:
# 2. Install requirements.txt, especially for the '"vllm" thing.
!pip install --upgrade pip
!pip install -r requirements.txt

In [ ]:
''' 3. Data 1:
Upload your zipped mt_bench2, containing 'questions.jsonl' (adversarial) and 'questions3' (old)
as set up before like in our local folder. Next we'll actually get 'questions.jsonl' where it has to go.
'''

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp '/content/drive/My Drive/question.jsonl' data/

In [ ]:
# MODEL ANSWER GENERATION
# FOR EACH CLOSED-SOURCE PROVIDER, SET API KEY SECRET.

import os

os.environ['OPENAI_API_KEY'] = ''

In [ ]:
# Model Answer Generation (Proprietary)

# List of models for answer generation
closed_models = [
    {'model_path': 'openai/gpt-3.5-turbo', 'model_id': 'gpt-3.5-turbo'}
]

# Loop through each model and generate responses
for model in closed_models:
    !python llm_judge/gen_model_answer.py \
        --model-path {model['model_path']} \
        --model-id {model['model_id']} \
        --bench-name mt_bench

In [ ]:
# Model Answer Generation (Open)

open_models = [
    {'model_path': 'lmsys/vicuna-7b-v1.5', 'model_id': 'vicuna-7b-v1.5'},
    {'model_path': 'microsoft/Phi-3-mini-4k-instruct', 'model_id': 'Phi-3-mini-4k-instruct'},
    {'model_path': 'lmsys/fastchat-t5-3b-v1.0', 'model_id': 'fastchat-t5-3b-v1.0'},
    {'model_path': 'THUDM/chatglm-6b', 'model_id': 'chatglm-6b'}
]

for model in open_models:
    !python llm_judge/gen_model_answer.py \
        --model-path {model['model_path']} \
        --model-id {model['model_id']} \
        --bench-name mt_bench

In [ ]:
# After answers generated

!python llm_judge/assign_judge.py

In [ ]:
!mv llm_judge/data/judge_prompts.jsonl data/

In [ ]:
# Judgment of Pairs; Specify (real, existing on HuggingFace) Judge Model Name or Names.
# Error: judge model smth smth
# Error: no judge_prompts file in Data
# Error:

!export OPENAI_API_KEY=''
!python llm_judge/gen_judgment.py \
--mode pairwise-all \
--new-judge-model gpt-3.5-turbo \
--model-list fastchat-t5-3b-v1.0 vicuna-7b-v1.5 gpt-3.5-turbo \
--batch-size 512 \
--bench-name mt_bench


In [ ]:
# Ablation
!ls
!ls data/mt_bench/model_judgment
!cd con_optimization && python main_ablation.py --baseline 0 --mode Order --epoch 30

In [ ]:
!zip -r /content/final_project_int.zip /content/Peer-review-in-LLMs

In [ ]:
from google.colab import files
files.download('/content/final_project.zip')